<a href="https://colab.research.google.com/github/lyloc-logtech/ai-code/blob/main/chatbottheokeyword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import re

# Đọc dữ liệu sản phẩm
df = pd.read_csv("/content/Bảng tính không có tiêu đề - Trang tính1.csv")
df["Tên sản phẩm"] = df["Tên sản phẩm"].str.lower()
df["Giới tính"] = df["Giới tính"].str.lower()

# Ghi nhớ trạng thái trò chuyện
user_gender = None
last_product_type = None

# Từ khóa loại sản phẩm
keywords = {
    "áo": ["áo", "hoodie", "croptop", "khoác", "polo", "sơ mi"],
    "quần": ["quần", "jean", "tây", "short", "kaki"],
    "váy": ["váy", "đầm"]
}

def get_product_type(query):
    for type_, words in keywords.items():
        if any(word in query for word in words):
            return type_
    return None

def extract_physical_info(query):
    cao_match = re.search(r"(\d{1,2}(?:\.\d{1,2})?)m", query)
    nang_match = re.search(r"(\d{2,3})kg", query)
    if cao_match and nang_match:
        cao = float(cao_match.group(1)) * 100  # m -> cm
        nang = int(nang_match.group(1))
        return cao, nang
    return None, None

def suggest_size(query, gender, product_type):
    cao, nang = extract_physical_info(query)
    if not gender:
        return "Bạn vui lòng cho biết giới tính (nam hoặc nữ) để tôi tư vấn size chính xác hơn."
    if not product_type:
        return "Bạn muốn tư vấn size cho áo, quần hay váy ạ?"

    if cao and nang:
        if gender == "nam":
            if product_type == "áo":
                if cao < 165 and nang <= 58:
                    return "Bạn là nam, nên mặc áo size S."
                elif cao < 170 and nang <= 66:
                    return "Bạn là nam, nên mặc áo size M."
                elif cao < 175 and nang <= 74:
                    return "Bạn là nam, nên mặc áo size L."
                elif cao < 180 and nang <= 82:
                    return "Bạn là nam, nên mặc áo size XL."
                else:
                    return "Bạn là nam, nên mặc áo size XXL."
            elif product_type == "quần":
                if cao < 165 and nang <= 58:
                    return "Bạn là nam, nên mặc quần size 28–29."
                elif cao < 170 and nang <= 66:
                    return "Bạn là nam, nên mặc quần size 30–31."
                elif cao < 175 and nang <= 74:
                    return "Bạn là nam, nên mặc quần size 32–33."
                else:
                    return "Bạn là nam, nên mặc quần size 34+."
        elif gender == "nữ":
            if product_type == "áo":
                if cao < 158 and nang <= 47:
                    return "Bạn là nữ, nên mặc áo size S."
                elif cao < 164 and nang <= 54:
                    return "Bạn là nữ, nên mặc áo size M."
                elif cao < 170 and nang <= 62:
                    return "Bạn là nữ, nên mặc áo size L."
                elif cao < 175 and nang <= 70:
                    return "Bạn là nữ, nên mặc áo size XL."
                else:
                    return "Bạn là nữ, nên mặc áo size XXL."
            elif product_type in ["quần", "váy"]:
                if cao < 158 and nang <= 47:
                    return "Bạn là nữ, nên mặc size 25–26."
                elif cao < 164 and nang <= 54:
                    return "Bạn là nữ, nên mặc size 27–28."
                elif cao < 170 and nang <= 62:
                    return "Bạn là nữ, nên mặc size 29–30."
                else:
                    return "Bạn là nữ, nên mặc size 31+."

    return "Xin lỗi, tôi chưa đủ thông tin để tư vấn size."

def find_products(query, gender):
    results = []
    for _, row in df.iterrows():
        if gender and gender not in row["Giới tính"] and row["Giới tính"] != "unisex":
            continue
        if any(word in query for word in row["Tên sản phẩm"].split()):
            results.append(f'{row["Tên sản phẩm"].title()}: {row["Mô tả"]}. Giá: {row["Giá tiền"]}')
    return results

def chatbot(query):
    global user_gender, last_product_type
    query = query.lower().strip()

    if "tôi là nam" in query or query in ["nam"]:
        user_gender = "nam"
        return "Bạn là nam, tôi đã ghi nhớ. Bạn muốn xem sản phẩm hay tư vấn size ạ?"
    elif "tôi là nữ" in query or query in ["nữ"]:
        user_gender = "nữ"
        return "Bạn là nữ, tôi đã ghi nhớ. Bạn muốn xem sản phẩm hay tư vấn size ạ?"

    if "m" in query and "kg" in query:
        product_type = get_product_type(query) or last_product_type
        return suggest_size(query, user_gender, product_type)

    found_type = get_product_type(query)
    if found_type:
        last_product_type = found_type
    products = find_products(query, user_gender)
    if products:
        return "\n\n".join(products)

    return "Xin lỗi, tôi chưa rõ yêu cầu của bạn. Bạn có thể hỏi về sản phẩm hoặc nhờ tư vấn size nhé."

# Giao diện trò chuyện
print("👕 Xin chào! Tôi là chatbot thời trang.")
print("Bạn có thể hỏi về sản phẩm hoặc tư vấn size (VD: 'Tôi là nữ, cao 1m60, nặng 50kg, muốn mua váy')")
print("Gõ 'thoát' để kết thúc.\n")

while True:
    user_input = input("Bạn: ")
    if user_input.lower() in ["thoát", "exit", "quit"]:
        print("👋 Tạm biệt! Hẹn gặp lại.")
        break
    print("Chatbot:", chatbot(user_input))


👕 Xin chào! Tôi là chatbot thời trang.
Bạn có thể hỏi về sản phẩm hoặc tư vấn size (VD: 'Tôi là nữ, cao 1m60, nặng 50kg, muốn mua váy')
Gõ 'thoát' để kết thúc.

Bạn: nam
Chatbot: Bạn là nam, tôi đã ghi nhớ. Bạn muốn xem sản phẩm hay tư vấn size ạ?
Bạn: áo
Chatbot: Áo Hoodie Basic: Áo hoodie vải nỉ, phong cách năng động. Giá: 250000
Bạn: cao 1m60,nang 50kg
Chatbot: Bạn là nam, nên mặc áo size S.
Bạn: thoát
👋 Tạm biệt! Hẹn gặp lại.
